In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import sys
sys.path.append("..")

# Start loading the data and getting the vocabulary

In [3]:
df = pd.read_parquet("../data/test.parquet")

In [4]:
vocabulary = set([x for xs in df['elements_1'] for x in xs])

# Check one list contains another list

## Brute force

In [5]:
from fast_filters.brute_force import brute_force_c2_contains_c1

In [6]:
%%time
index_brute = brute_force_c2_contains_c1(df, 'elements_1', 'elements_2')

CPU times: user 10.9 s, sys: 125 ms, total: 11 s
Wall time: 11.1 s


## Binary trick

In [7]:
from fast_filters.binary import Converter, vectorized_not_null_intersection, vectorized_c2_contains_c1

In [8]:
%%time
converter = Converter(vocabulary=vocabulary)
df['elements_1_bin'] = df['elements_1'].map(converter.convert)
df['elements_2_bin'] = df['elements_2'].map(converter.convert)
index_vec = vectorized_not_null_intersection(df, 'elements_1_bin', 'elements_2_bin')

CPU times: user 3.54 s, sys: 52.4 ms, total: 3.6 s
Wall time: 3.63 s


In [9]:
%%time
index_vec = vectorized_c2_contains_c1(df, 'elements_1_bin', 'elements_2_bin')

CPU times: user 3.61 ms, sys: 1.6 ms, total: 5.21 ms
Wall time: 4.05 ms


### how does the integer representation look like?

In [10]:
df.head()

,id,elements_1,elements_2,elements_1_bin,elements_2_bin
0,0,[gzvlwslrnh],[gzvlwslrnh],8,8
1,1,"[wsvkyudbfd, gpxgwigfvd, edjjnoqjyi]","[hdqfajwgbw, wsvkyudbfd, vtqldagyxl]",6146,28672
2,2,[vtqldagyxl],"[wdhltmuqti, edjjnoqjyi]",8192,1026
3,3,"[oyizmzfmpr, hdqfajwgbw, wsvkyudbfd, edjjnoqjyi]","[xwyehwnnbw, xczmujmhug, owetlrgmyw, gpxgwigfvd]",20486,2704
4,4,"[xwyehwnnbw, edjjnoqjyi]","[owetlrgmyw, wdhltmuqti]",18,1152


# Final checks

In [11]:
all(index_brute == index_vec)

True